In [1]:
from tqdm.notebook import tqdm

In [2]:
import requests
from bs4 import BeautifulSoup

### 1. KOBIS 영화관 입장권 통합 전산망에서 영화 제목 크롤링

총 10,000개의 영화 제목을 수집합니다.

In [3]:
URL = "http://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do"

In [4]:
page = 5
payload = {'curPage':page}

In [5]:
r = requests.get(URL, params = payload)

In [6]:
soup = BeautifulSoup(r.text, "html.parser")

In [114]:
movie_title = set()

In [172]:
for page_num in tqdm(range(10)):
    r = requests.get(URL, params = {"curPage":page_num+1})
    soup = BeautifulSoup(r.text, "html.parser")
    for i in range(10):
        tmp = str(soup.select("#content > div.rst_sch > table > tbody > tr:nth-of-type("+str(i+1)+") > td:nth-of-type(1)")[0].a)
        movie_title.add(tmp.split(">")[1].split("<")[0])

In [118]:
len(movie_title)

10

In [119]:
movie_title

{'가장 보통의 연애',
 '마음 울적한 날엔',
 '메기',
 '버티고',
 '사선의 끝',
 '수상한 이웃',
 '열두 번째 용의자',
 '장사리 : 잊혀진 영웅들',
 '첫잔처럼',
 '판소리 복서'}

### 2. 네이버 영화에서 별점과 댓글 크롤링

In [169]:
import time
import random

In [171]:
time.sleep(random.random()*2)

In [172]:
star_rate = []

In [9]:
title = "가장 보통의 연애"

In [99]:
naver = []

In [170]:
code = ""
for i in str(title.encode('euc-kr')).split("\\x")[1:]:
    code += "%"+i.upper().replace(" ","+")
url = "https://movie.naver.com/movie/search/result.nhn?query=" + code + "&section=all"
r = requests.get(url)
soup = BeautifulSoup(r.text, "html.parser")
movie_code = str(soup.select("#old_content > ul.search_list_1 > li > p > a")[0]).split("=")[2][:6]
movie_code

# 네이버 고유 영화 코드 찾기
r = requests.get(url)
soup = BeautifulSoup(r.text, "html.parser")
movie_code = str(soup.select("#old_content > ul.search_list_1 > li > p > a")[0]).split("=")[2][:6]
url = "https://movie.naver.com/movie/bi/mi/point.nhn?code="+movie_code+"&onlyActualPointYn=Y#pointAfterTab"

# 평점 정보 수집하기
r = requests.get(url)
soup = BeautifulSoup(r.text, "html.parser")
rate = round(float(str(soup.select("#actual_point_tab_inner > div > span > span")[0]).split(":")[1].split("%")[0]) / 10.00, 3)
people_num = int(soup.select("#graph_area > div.grade_netizen > div.title_area.grade_tit > div.sc_area > span > em")[0].text.replace(",",""))

# 평점이 너무 적은 데이터는 표본에서 제거
if people_num < 5:
    movie.title.discard(title)
else:
    naver.append((title, rate, people_num))

# 상위 3개 리뷰 가져오기
tmplst = []
url = "https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code="+movie_code+"&type=after&onlyActualPointYn=Y&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false"
r = requests.get(url)
soup = BeautifulSoup(r.text, "html.parser")
for i in range(3):
    s = soup.select("body > div > div > div.score_result > ul > li:nth-of-type("+str(i+1)+")")[0]
    top_rate = int(float(str(s.span.span).split(":")[1].split("%")[0])//10)
    top_review = soup.select("#_filtered_ment_"+str(i))[0].text.strip()
    tmplst.append((top_rate,top_review))


https://movie.naver.com/movie/bi/mi/point.nhn?code=182205&onlyActualPointYn=Y#pointAfterTab


In [ ]:
#content > div.article > div.section_group.section_group_frst > div.obj_section.noline > div > div.lst_people_area.height100 > ul > li:nth-child(1)

In [207]:
url = "https://movie.naver.com/movie/bi/mi/detail.nhn?code=126389"
r = requests.get(url)
soup = BeautifulSoup(r.text, "html.parser")

In [209]:
for i in range(8):
    try:
        print(str(soup.select("#content > div.article > div.section_group.section_group_frst > div.obj_section.noline > div > div.lst_people_area.height100 > ul > li:nth-of-type("+str(i+1)+") > p > a")[0]).split('"')[3])
    except:
        break

구윤희
양병간


In [171]:
tmplst

[(10,
  '연애와 직장, 모든것에 공감하며 미친듯이 웃다가- 현실적인 대사들에 뼈 후려맞고 살짝 아파하는 나를 발견...☆ 최근 본 코미디/로맨스 장르 영화 중 가장 잘 나온 듯'),
 (10,
  '소오올찍히 그냥 저냥한 로코물인줄 알았는데 왜 공효진이 로코장인이라고 불리는지 또 깨닫게 해줌. 공효진 김래원 케미가 상상이 안됐는데 김래원의 재발견임ㅋㅋㅋ쓰면서도 웃김 김래원이 자꾸 생각나섴ㅋㅋㅋ간만에 볼거없는 ...'),
 (10,
  '공효진이랑 김래원 케미 미쳤다 ㅋㅋㅋㅋ 입모양게임하는거 보고 웃겨 죽을뻔 ㅠㅠ 팍팍한 삶에 한줄기 빛과 소금같은 영화였습니다...☆')]

In [65]:
import re
import json
import urllib

In [122]:
url = "https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code="+movie_code+"&type=after&onlyActualPointYn=Y&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false"

In [126]:
r = requests.get(url)
soup = BeautifulSoup(r.text, "html.parser")

In [173]:
for i in tqdm(range(30)):
    try:
        title = movie_data[i][1]
        # 해당하는 영화의 url 찾기
        code = ""
        for i in str(title.encode('euc-kr')).split("\\x")[1:]:
            code += "%"+i.upper().replace(" ","+")
        url = "https://movie.naver.com/movie/search/result.nhn?query=" + code + "&section=all"
        # 네이버 고유 영화 코드 찾기
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")
        movie_code = str(soup.select("#old_content > ul.search_list_1 > li > p > a")[0]).split("=")[2][:6]
        url = "https://movie.naver.com/movie/bi/mi/point.nhn?code="+movie_code+"&onlyActualPointYn=Y#pointAfterTab"
        # 평점 정보 수집하기
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")
        rate = float(str(soup.select("#actual_point_tab_inner > div > span > span")[0]).split(":")[1].split("%")[0]) / 10
        if rate:
            print(i, title, rate)
            star_rate.append((title, rate))
    except:
        continue

d6' 가장 보통의 연애 8.540000000000001
e2' 메기 8.78
a3' 마음 울적한 날엔 9.5
ba' 디어 마이 지니어스 10.0
d2' 더 프롬 10.0



In [174]:
star_rate

[('가장 보통의 연애', 8.540000000000001),
 ('메기', 8.78),
 ('마음 울적한 날엔', 9.5),
 ('디어 마이 지니어스', 10.0),
 ('더 프롬', 10.0)]

### 3. 다음 영화에서 별점과 댓글 크롤링